In [1]:
import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import numpy as np
from torch.nn import functional as f
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn import svm
from sklearn.metrics import accuracy_score

In [2]:
dataset = MNIST('./data', download=True)
dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train

In [3]:
C = 1.0
train_size = 5000
test_size = 1000

# SVM on raw flattened data

In [4]:
X = np.array(dataset.data)
X = X.reshape(len(X), 28*28)
Y = np.array(dataset.targets)
print(X.shape, Y.shape)

(60000, 784) (60000,)


In [5]:
train_X = X[:train_size]
train_Y = Y[:train_size]

test_X = X[train_size: train_size + test_size]
test_Y = Y[train_size: train_size + test_size]

clf = svm.SVC(C=C)
clf.fit(train_X, train_Y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [6]:
accuracy_score(clf.predict(test_X), test_Y)

0.954

# SVM on localized autoencoder

In [7]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


In [8]:
model = VAE()
model.load_state_dict(torch.load('Localized_AE.pth'))

<All keys matched successfully>

In [9]:
img = dataset.data
img = img.reshape(len(img), 1, 28, 28).float()
img = img.view(img.size(0), -1)
img = Variable(img)
mu, logvar = model.encode(img)
X = model.reparametrize(mu, logvar).detach().numpy()

In [10]:
train_X = X[:train_size]
train_Y = Y[:train_size]

test_X = X[train_size: train_size + test_size]
test_Y = Y[train_size: train_size + test_size]

clf = svm.SVC(C=C)
clf.fit(train_X, train_Y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [11]:
accuracy_score(clf.predict(test_X), test_Y)

0.97

# SVM on normal autoencoder

In [12]:
model = VAE()
model.load_state_dict(torch.load('AE.pth'))

<All keys matched successfully>

In [13]:
img = dataset.data
img = img.reshape(len(img), 1, 28, 28).float()
img = img.view(img.size(0), -1)
img = Variable(img)
mu, logvar = model.encode(img)
X = model.reparametrize(mu, logvar).detach().numpy()

In [14]:
train_X = X[:train_size]
train_Y = Y[:train_size]

test_X = X[train_size: train_size + test_size]
test_Y = Y[train_size: train_size + test_size]

clf = svm.SVC(C=C)
clf.fit(train_X, train_Y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [15]:
accuracy_score(clf.predict(test_X), test_Y)

0.964